In [1]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
from torchinfo import summary
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import torchvision.transforms as T
from glob import glob
import segmentation_models_pytorch as smp
import torch.nn.functional as F
from collections import defaultdict
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
tf=T.ToTensor()


In [2]:
device

device(type='cuda', index=5)

In [2]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':16,
        'epochs':500,}

In [3]:
image_path='../../data/external/ori/*.png'
mask1_path='../../data/external/mask/class1/*.png'

In [4]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path = Image.open(self.img_path[idx])
        image_path=tf(image_path)
        file_path=os.path.basename(self.img_path[idx])
        label1 = np.array(Image.open(self.label[idx]))
        label1=label1[:,:,0,np.newaxis]
        label2=np.array(Image.open(self.label[idx].replace('/class1', '/class2')))
        label2=label2[:,:,0,np.newaxis]
        label3=np.array(Image.open(self.label[idx].replace('/class1', '/class3')))
        label3=label3[:,:,0,np.newaxis]

        label=np.concatenate((label1,label2,label3),axis=2)
        label_path = tf(cv2.resize(label, (512, 512)))
       
        return image_path, label_path,file_path

test_dataset = CustomDataset(glob(image_path), glob(mask1_path))
test_dataloader = DataLoader(
    test_dataset, batch_size=params['batch_size'],shuffle=False, drop_last=True)


In [5]:
def dice_loss(pred, target, num_classes=3):
    smooth = 0
    dice_per_class = torch.zeros(num_classes).to(pred.device)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        A_sum = torch.sum(pred_class * pred_class)
        B_sum = torch.sum(target_class * target_class)

        dice_per_class[class_id] =(2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return dice_per_class

def compute_iou(pred_mask, true_mask, threshold=0.5, num_classes=3):
    """
    IoU를 계산하는 함수

    :param pred_mask: 모델이 예측한 마스크 (torch.Tensor)
    :param true_mask: 실제 마스크 (torch.Tensor)
    :param threshold: 이진화를 위한 임계값
    :return: IoU 값
    """
    iou_per_class = torch.zeros(num_classes).to(device)
    for class_id in range(num_classes):
    # 예측된 마스크 이진화
        pred_mask1 = (pred_mask[:,class_id, ...] > threshold).float()
        
        # 실제 마스크 이진화
        true_mask1 = (true_mask[:,class_id, ...] > threshold).float()
        
        # 교차 계산
        intersection = torch.sum(pred_mask1 * true_mask1)
        
        # 합집합 계산
        union = torch.sum(pred_mask1) + torch.sum(true_mask1) - intersection
        
        # IoU 계산
        iou_per_class[class_id]= intersection / union
    
    return iou_per_class

def compute_f1(pred_mask, true_mask, threshold=0.5, num_classes=3):
    """
    F1 점수를 계산하는 함수

    :param pred_mask: 모델이 예측한 마스크 (torch.Tensor)
    :param true_mask: 실제 마스크 (torch.Tensor)
    :param threshold: 이진화를 위한 임계값
    :param num_classes: 클래스의 수
    :param device: 연산에 사용할 디바이스 (기본값: 'cpu')
    :return: 각 클래스별 F1 점수 (torch.Tensor)
    """
    f1_per_class = torch.zeros(num_classes).to(device)
    precision1=torch.zeros(num_classes).to(device)
    recall1=torch.zeros(num_classes).to(device)
    
    for class_id in range(num_classes):
        # 예측된 마스크 이진화
        pred_binary_mask = (pred_mask[:, class_id, ...] > threshold).float()
        
        # 실제 마스크 이진화
        true_binary_mask = (true_mask[:, class_id, ...] > threshold).float()
        
        # True Positive (TP), False Positive (FP), False Negative (FN) 계산
        TP = torch.sum(pred_binary_mask * true_binary_mask)
        FP = torch.sum(pred_binary_mask * (1 - true_binary_mask))
        FN = torch.sum((1 - pred_binary_mask) * true_binary_mask)
        
        # 정밀도 (Precision) 계산
        precision = TP / (TP + FP + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # 재현율 (Recall) 계산
        recall = TP / (TP + FN + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # F1 점수 계산
        f1_per_class[class_id] = 2 * (precision * recall) / (precision + recall + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        precision1[class_id]=precision.item()
        recall1[class_id]=recall.item()
    
    return f1_per_class,precision,recall

class MultiClassIoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(MultiClassIoULoss, self).__init__()

    def forward(self, inputs, targets,num_classes=3, smooth=1):
        num_classes = inputs.shape[1]  # assuming inputs have shape (batch_size, num_classes, height, width)
        inputs = F.softmax(inputs, dim=1)  # apply softmax to the input tensor
        IOU_per_class = torch.zeros(num_classes).to(device)
        for cls in range(num_classes):
            input_cls = inputs[:, cls, :, :].contiguous().view(-1)  # flatten the inputs for the current class
            target_cls = targets[:, cls, :, :].float().contiguous().view(-1)  # create binary target for the current class

            intersection = (input_cls * target_cls).sum()
            total = (input_cls + target_cls).sum()
            union = total - intersection

            IoU = (intersection + smooth) / (union + smooth)
            IOU_per_class[cls] = (1 - IoU)

        return IOU_per_class # average over all classes
    
model = smp.MAnet(
    encoder_name="efficientnet-b5",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
).to(device)
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=params['lr'], betas=(params['beta1'], params['beta2']))
criterion = MultiClassIoULoss()

In [ ]:
for i in range(2):
    model.load_state_dict(torch.load('../../model/MANet/MANet_'+str(i+1)+'_check.pth',map_location=device))
    df=pd.DataFrame(columns=['file_name','Dice1','Dice2','Dice3','mDice','IoU1','IoU2','IoU3','mIoU','f1','precision','recall'])
    with torch.no_grad():
        test = tqdm(test_dataloader)
        count = 0
        val_running_loss = 0.0
        acc_loss = 0
        for x, y,file_path in test:
            model.eval()
            y = y.to(device).float()
            count += 1
            x = x.to(device).float()
            predict = model(x).to(device)
            cost = 1-compute_iou(y,predict)  # cost 구함
            iou=compute_iou(predict, y)
            f1,precision,recall=compute_f1(predict, y)
            val_running_loss+=cost.mean().item()
            df.loc[len(df)]=[file_path[0],cost[0].item(),cost[1].item(),cost[2].item(),cost.mean().item(),iou[0].item(),iou[1].item(),iou[2].item(),iou.mean().item(),f1.mean().item(),precision.mean().item(),recall.mean().item()]
            
            test.set_description(
                f"val_Step: {count+1} dice_loss : {val_running_loss/count:.4f}")
    df.to_csv('../../data/external/result/MANet/MANet_'+str(i+1)+'_result.csv',index=False)

In [6]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
from torchinfo import summary
import torch
import numpy as np
import torch.optim as optim
import torchvision.transforms as T

import segmentation_models_pytorch as smp
import torch.nn.functional as F
from collections import defaultdict
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
tf=T.ToTensor()
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':32,
        'epochs':100,}
image1=np.load('../../data/cv0_ori.npy')
image1=image1.astype(np.uint8)
image2=np.load('../../data/cv1_ori.npy')
image2=image2.astype(np.uint8)
image3=np.load('../../data/cv2_ori.npy')
image3=image3.astype(np.uint8)
image4=np.load('../../data/cv3_ori.npy')
image4=image4.astype(np.uint8)
image5=np.load('../../data/cv4_ori.npy')
image5=image5.astype(np.uint8)
mask1=np.load('../../data/cv0_mask.npy')
mask1=(mask1[:,:,:,:3]).astype(np.uint8)
mask2=np.load('../../data/cv1_mask.npy')
mask2=(mask2[:,:,:,:3]).astype(np.uint8)
mask3=np.load('../../data/cv2_mask.npy')
mask3=(mask3[:,:,:,:3]).astype(np.uint8)
mask4=np.load('../../data/cv3_mask.npy')
mask4=(mask4[:,:,:,:3]).astype(np.uint8)
mask5=np.load('../../data/cv4_mask.npy')
mask5=(mask5[:,:,:,:3]).astype(np.uint8)

np_data={'image1':image1,'image2':image2,'image3':image3,'image4':image4,'image5':image5,'mask1':mask1,'mask2':mask2,'mask3':mask3,'mask4':mask4,'mask5':mask5}

class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path = self.img_path[idx]
        image_path=tf(cv2.cvtColor(image_path, cv2.COLOR_GRAY2RGB))
        
        label_path = self.label[idx]
        label_path = tf(cv2.resize(label_path[:,:,:3], (512, 512)))
       
        return image_path, label_path
    
def dice_loss(pred, target, num_classes=3):
    smooth = 1.
    dice_per_class = torch.zeros(num_classes).to(pred.device)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        A_sum = torch.sum(pred_class * pred_class)
        B_sum = torch.sum(target_class * target_class)

        dice_per_class[class_id] = 1 - \
            (2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return torch.mean(dice_per_class)
def iou_loss(pred, target, num_classes=3):
    smooth = 1e-6
    iou_per_class = torch.zeros(num_classes).to(pred.device)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        union = torch.sum(pred_class) + torch.sum(target_class) - intersection

        iou_per_class[class_id] = 1 - (intersection + smooth) / (union + smooth)

    return torch.mean(iou_per_class)
    
metrics = defaultdict(float)
for k in range(2,5):
    val_loss=1000
    df=pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss', 'train_acc', 'val_acc'])
    train_list=[0,1,2,3,4]
    train_list.remove(k)
    train_image=np.concatenate([np_data['image'+str(i+1)] for i in train_list])
    train_mask=np.concatenate([np_data['mask'+str(i+1)] for i in train_list])
    val_image=np_data['image'+str(k+1)]
    val_mask=np_data['mask'+str(k+1)]
    train_dataset = CustomDataset(train_image, train_mask)

    val_dataset = CustomDataset(val_image, val_mask)
    train_dataloader = DataLoader(
    train_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)
    validation_dataloader = DataLoader(
    val_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)
    ealry_count=0
    model = smp.MAnet(
        encoder_name="efficientnet-b5",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=3,                      # model output channels (number of classes in your dataset) 
    ).to(device)
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), lr=params['lr'], betas=(params['beta1'], params['beta2']))
    for epoch in range(100):
        train = tqdm(train_dataloader)
        count = 0
        running_loss = 0.0
        acc_loss = 0
        for x, y in train:
            model.train()
            y = y.to(device).float()
            count += 1
            x = x.to(device).float()
            optimizer.zero_grad()  # optimizer zero 로 초기화
            predict = model(x).to(device)
            cost = iou_loss(predict, y)  # cost 구함
            acc = 1-cost.item()
            cost.backward()  # cost에 대한 backward 구함
            optimizer.step()
            running_loss += cost.item()
            acc_loss += acc
            train.set_description(
                f"epoch: {epoch+1}/{300} Step: {count+1} iou_loss : {running_loss/count:.4f} iou_score: {1-running_loss/count:.4f}")
            
        with torch.no_grad():
            val = tqdm(validation_dataloader)
            count = 0
            val_running_loss = 0.0
            acc_loss = 0
            for x, y in val:
                model.eval()
                y = y.to(device).float()
                count += 1
                x = x.to(device).float()
                predict = model(x).to(device)
                cost= iou_loss(predict, y)  # cost 구함
                acc = 1-cost.item()
                val_running_loss += cost.item()
                acc_loss += acc
                val.set_description(
                    f"val_epoch: {epoch+1}/{300} Step: {count+1} iou_loss : {val_running_loss/count:.4f} iou_score: {1-val_running_loss/count:.4f}")
        if val_loss>(val_running_loss/count):
            ealry_count=0
            val_loss=val_running_loss/count
            torch.save(model.state_dict(), '../../model/MANet/MANet_'+str(k+1)+'_check.pth')
        else:
            ealry_count+=1
            if epoch>10 and ealry_count==3:
                break
        df.loc[len(df)]=[epoch+1,running_loss/len(train_dataloader),val_running_loss/len(validation_dataloader),1-running_loss/len(train_dataloader),1-val_running_loss/len(validation_dataloader)]
        df.to_csv('../../model/MANet/MANet_'+str(k+1)+'.csv',index=False)
    torch.save(model.state_dict(), '../../model/MANet/MANet_'+str(k+1)+'.pth')
    

epoch: 1/300 Step: 15 iou_loss : 0.9388 iou_score: 0.0612:   5%|▌         | 14/256 [00:19<05:40,  1.41s/it]


KeyboardInterrupt: 

In [7]:
cost

tensor([ 0.0226,  0.0031, -0.0097], device='cuda:5', grad_fn=<CopySlices>)